In [67]:
import numpy as np
import healpy as hp
import pymaster as nmt
from scipy import optimize

In [25]:
nside=256
freqs=np.arange(544,1088,step=1)
# nfreqs=len(freqs)
nfreqs=181
lmax=3*nside -1
almsize=hp.Alm.getsize(lmax)

In [20]:
lmax

767

In [26]:
almsize

295296

In [ ]:
smoothed_hi_maps=np.load("smoothed_hi_maps.npy") #first chunk nf=181
smoothed_maps= np.load("smoothed_maps_1_smallbeam.npy") #first chunk
noise_maps= np.load("noise_nf.npy")

In [15]:
noise_maps_1=noise_maps[:181]

In [17]:
noise_maps_1.shape


(181, 786432)

In [14]:
hp.nside2npix(nside)

786432

We observe $$x_\ell = s_\ell + n_\ell + f_\ell$$
The Model Covariance is written as:
$$C_\ell^{\text{model}} = C_{\text{HI}} + C_{\text{noise}} + F P_\ell F^T$$
Where:

- $F \in \mathbb{R}^{N_c \times r}$: foreground mixing matrix (unknown)
- $P_\ell \in \mathbb{R}^{r \times r}$: power spectra of foreground components at multipole ℓ (unknown)
- $r≪N_c$: the rank of the foreground model

In [29]:
def compute_cl(almmaps:np.ndarray, l_max: int, n_freq: int)-> tuple[np.ndarray, np.ndarray]:
    r"""
    Compute the power spectrum C_l from alm_maps.

    The power spectrum is computed as:

    \[ \hat{C}_{l}^{ij} = \frac{1}{2l+1}\sum_{m=-l}^{+l} a_{lm}^{i}a^{j \dagger }_{lm} \]
    
    Parameters:
    ----------
    almmaps : np.ndarray
        Array of shape (n_freq, num_alm) containing spherical harmonic coefficients.
    
    l_max : int
        Maximum multipole lmax.
    
    n_freq : int
        Number of frequency channels.
    
    Returns:
    -------
    alm_p : np.ndarray
        Truncated spherical harmonic values for m>0.
    
    C_l : np.ndarray
        Power spectrum array of shape (lmax+1, n_freq, n_freq).
    """
    Cl = np.zeros((l_max + 1, n_freq, n_freq), dtype=float)
    
    for l in range(l_max + 1):
        
        index = np.zeros((l + 1), dtype=int)
        
        for m in range(l + 1):
            index[m] = hp.Alm.getidx(l_max, l, m)  # Get Alm index
        
        almp = almmaps[:, index]  # Extract relevant alm values
        print(f"Computing power spectra {l}")
        # Compute C_l
        Cl[l, :, :] = np.real(np.outer(almp[:, 0], almp[:, 0]))
        
        for m in range(1, l + 1):  # Sum over m
            Cl[l, :, :] += 2 * np.real(np.outer(almp[:, m], np.conj(almp[:, m])))
    
    return Cl

In [53]:
def compute_model_covariance(maps, lmax, almsize, nfreqs):
    alms=np.zeros((nfreqs, almsize), dtype=complex) 
    
    for nf in range(nfreqs):
        print(f"Converting maps {nf}")
        alms[nf,:]= hp.map2alm(maps[nf,:], lmax=lmax) 
        
    cl=compute_cl(alms, lmax, nfreqs)
    
    return cl

In [54]:
# Getting the model covariance of HI and noise which we assume to be known
# F and P_l are free parameters to be estimated

cl_noise= compute_model_covariance(noise_maps_1, lmax, almsize, nfreqs)
cl_hi= compute_model_covariance(smoothed_hi_maps, lmax, almsize, nfreqs)

Converting maps 0
Converting maps 1
Converting maps 2
Converting maps 3
Converting maps 4
Converting maps 5
Converting maps 6
Converting maps 7
Converting maps 8
Converting maps 9
Converting maps 10
Converting maps 11
Converting maps 12
Converting maps 13
Converting maps 14
Converting maps 15
Converting maps 16
Converting maps 17
Converting maps 18
Converting maps 19
Converting maps 20
Converting maps 21
Converting maps 22
Converting maps 23
Converting maps 24
Converting maps 25
Converting maps 26
Converting maps 27
Converting maps 28
Converting maps 29
Converting maps 30
Converting maps 31
Converting maps 32
Converting maps 33
Converting maps 34
Converting maps 35
Converting maps 36
Converting maps 37
Converting maps 38
Converting maps 39
Converting maps 40
Converting maps 41
Converting maps 42
Converting maps 43
Converting maps 44
Converting maps 45
Converting maps 46
Converting maps 47
Converting maps 48
Converting maps 49
Converting maps 50
Converting maps 51
Converting maps 52
Con

Diagonalize the power spectra, so we have $C^{ij}(l)$ 

In [60]:
def diagonalize_matrices(cl, nfreqs):
    cl_diag=np.zeros((lmax+1, nfreqs, nfreqs))
    for l in range (lmax+1):
        print(f"Diagonalizing matrix {l}")
        cl_diag[l,:,:]= np.identity(nfreqs)*cl[l,:,:]
    return cl_diag

In [61]:
hi= diagonalize_matrices(cl_hi, nfreqs)
noise= diagonalize_matrices(cl_noise, nfreqs)

Diagonalizing matrix 0
Diagonalizing matrix 1
Diagonalizing matrix 2
Diagonalizing matrix 3
Diagonalizing matrix 4
Diagonalizing matrix 5
Diagonalizing matrix 6
Diagonalizing matrix 7
Diagonalizing matrix 8
Diagonalizing matrix 9
Diagonalizing matrix 10
Diagonalizing matrix 11
Diagonalizing matrix 12
Diagonalizing matrix 13
Diagonalizing matrix 14
Diagonalizing matrix 15
Diagonalizing matrix 16
Diagonalizing matrix 17
Diagonalizing matrix 18
Diagonalizing matrix 19
Diagonalizing matrix 20
Diagonalizing matrix 21
Diagonalizing matrix 22
Diagonalizing matrix 23
Diagonalizing matrix 24
Diagonalizing matrix 25
Diagonalizing matrix 26
Diagonalizing matrix 27
Diagonalizing matrix 28
Diagonalizing matrix 29
Diagonalizing matrix 30
Diagonalizing matrix 31
Diagonalizing matrix 32
Diagonalizing matrix 33
Diagonalizing matrix 34
Diagonalizing matrix 35
Diagonalizing matrix 36
Diagonalizing matrix 37
Diagonalizing matrix 38
Diagonalizing matrix 39
Diagonalizing matrix 40
Diagonalizing matrix 41
Di

Now we bin the power spectra, with $\Delta \ell=10$?

In [79]:
def bin_spectra(cl,nside, delta):
    b= nmt.NmtBin.from_lmax_linear(3*nside-1, nlb=delta)
    n_bins = b.get_n_bands()
    cl_binned= np.zeros((n_bins, nfreqs, nfreqs))
    for nf in range(nfreqs):
        cl_binned [:,nf, nf]= b.bin_cell(cl[:,nf,nf])
    return cl_binned

In [80]:
noise_binned= bin_spectra(noise, nside, 10)
hi_binned= bin_spectra(hi, nside, 10)